# Car Price Prediction::

Download dataset from this link:

https://www.kaggle.com/hellbuoy/car-price-prediction

# Problem Statement::

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


A Chinese automobile company Geely Auto aspires to enter the US market by setting up their manufacturing unit there and producing cars locally to give competition to their US and European counterparts.

They have contracted an automobile consulting company to understand the factors on which the pricing of cars depends. Specifically, they want to understand the factors affecting the pricing of cars in the American market, since those may be very different from the Chinese market. The company wants to know:

Which variables are significant in predicting the price of a car
How well those variables describe the price of a car
Based on various market surveys, the consulting firm has gathered a large data set of different types of cars across the America market.

# task::
We are required to model the price of cars with the available independent variables. It will be used by the management to understand how exactly the prices vary with the independent variables. They can accordingly manipulate the design of the cars, the business strategy etc. to meet certain price levels. Further, the model will be a good way for management to understand the pricing dynamics of a new market.

# WORKFLOW ::

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

4.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

5.Compilation Step (Note : Its a Regression problem , select loss , metrics according to it)
6.Train the Model with Epochs (100) and validate it

7.If the model gets overfit tune your model by changing the units , No. of layers , activation function , epochs , add dropout layer or add Regularizer according to the need .

8.Evaluation Step

9.Prediction

In [33]:
#import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras import models,layers,optimizers,utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [34]:
#load dataset
df=pd.read_csv("/content/drive/MyDrive/PIAIC_ASSIGNMENTS/CarPrice_Assignment.csv")

In [35]:
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


Check  MIssing Value

In [50]:
# creating a dataframe from list
df = pd.DataFrame(data)

#CHECK MISSING VALUE if exist replace it with mean
if df.isnull:
  df.fillna(df.mean())
  df.head()



In [36]:
#get data info to check object dataype 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [37]:
#drop car price, id and name 
x=df.drop(["price","car_ID","CarName"],axis=1).copy()
y=df["price"].copy()

In [38]:
{column: list(x[column].unique()) for column in x.columns if x.dtypes[column]=='object'}

{'aspiration': ['std', 'turbo'],
 'carbody': ['convertible', 'hatchback', 'sedan', 'wagon', 'hardtop'],
 'cylindernumber': ['four', 'six', 'five', 'three', 'twelve', 'two', 'eight'],
 'doornumber': ['two', 'four'],
 'drivewheel': ['rwd', 'fwd', '4wd'],
 'enginelocation': ['front', 'rear'],
 'enginetype': ['dohc', 'ohcv', 'ohc', 'l', 'rotor', 'ohcf', 'dohcv'],
 'fuelsystem': ['mpfi', '2bbl', 'mfi', '1bbl', 'spfi', '4bbl', 'idi', 'spdi'],
 'fueltype': ['gas', 'diesel']}

In [39]:
x["doornumber"]=x["doornumber"].apply(lambda x: 0 if x == 'two' else 1)

In [40]:
numeric_ordering ={'two':2,'three':3,'four':4,'five':5,'six':6,'eight':8,'twelve':12}

In [41]:
x["cylindernumber"] =x["cylindernumber"].replace(numeric_ordering)

In [42]:
def binary_encode(x, columns, positive_values):
    x =x.copy()
    for column, positive_value in zip(columns, positive_values):
        x[column] = x[column].apply(lambda x: 1 if x == positive_value else 0)
    return x

In [43]:
binary_features=['fueltype','aspiration','enginelocation',]
binary_positive_values=['diesel','turbo','front']
x=binary_encode(x,columns=binary_features,positive_values=binary_positive_values)

In [44]:
binary_features = ['fueltype','aspiration','enginelocation']
binary_positive_values=['diesel','turbo','front']
x=binary_encode(x,columns=binary_features,positive_values=binary_positive_values)

In [45]:
def onehot_encode(x, columns, prefixes):
    x = x.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(x[column], prefix=prefix)
        x=pd.concat([x, dummies], axis=1)
        x=x.drop(column, axis=1)
    return x

In [46]:
nominal_features=['carbody','drivewheel','enginetype','fuelsystem']
prefixes=['MK','BS','DW','ET','FS']
x=onehot_encode(x,columns=nominal_features,prefixes=prefixes)

In [47]:
x

,symboling,fueltype,aspiration,doornumber,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,cylindernumber,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,MK_convertible,MK_hardtop,MK_hatchback,MK_sedan,MK_wagon,BS_4wd,BS_fwd,BS_rwd,DW_dohc,DW_dohcv,DW_l,DW_ohc,DW_ohcf,DW_ohcv,DW_rotor,ET_1bbl,ET_2bbl,ET_4bbl,ET_idi,ET_mfi,ET_mpfi,ET_spdi,ET_spfi
0,3,0,0,0,0,88.6,168.8,64.1,48.8,2548,4,130,3.47,2.68,9.0,111,5000,21,27,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,3,0,0,0,0,88.6,168.8,64.1,48.8,2548,4,130,3.47,2.68,9.0,111,5000,21,27,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,0,94.5,171.2,65.5,52.4,2823,6,152,2.68,3.47,9.0,154,5000,19,26,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,2,0,0,1,0,99.8,176.6,66.2,54.3,2337,4,109,3.19,3.40,10.0,102,5500,24,30,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
4,2,0,0,1,0,99.4,176.6,66.4,54.3,2824,5,136,3.19,3.40,8.0,115,5500,18,22,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,0,0,1,0,109.1,188.8,68.9,55.5,2952,4,141,3.78,3.15,9.5,114,5400,23,28,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
201,-1,0,0,1,0,109.1,188.8,68.8,55.5,3049,4,141,3.78,3.15,8.7,160,5300,19,25,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
202,-1,0,0,1,0,109.1,188.8,68.9,55.5,3012,6,173,3.58,2.87,8.8,134,5500,18,23,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
203,-1,0,0,1,0,109.1,188.8,68.9,55.5,3217,6,145,3.01,3.40,23.0,106,4800,26,27,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

In [51]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,random_state=123)

In [52]:
scaler = StandardScaler()
scaler.fit(x_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [53]:
x_train =pd.DataFrame(scaler.transform(x_train),columns=x_train.columns)
x_test=pd.DataFrame(scaler.transform(x_test),columns=x_test.columns)

In [54]:
part_x_train,x_val,part_y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=2)

Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

In [59]:
model = Sequential()
model.add(Dense(12, input_dim=42, kernel_initializer='normal', activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

Compilation Step (Note : Its a Regression problem , select loss , metrics according to it) 6.Train the Model with Epochs (100) and validate it

In [60]:
history=model.fit(part_x_train,part_y_train,epochs=100,validation_data=(x_val,y_val),batch_size=512)

Epoch 1/100
1/1 [==============================] - 1s 764ms/step - loss: 263509008.0000 - mse: 263509008.0000 - mae: 14128.1934 - val_loss: 334190208.0000 - val_mse: 334190208.0000 - val_mae: 14966.6465
Epoch 2/100
1/1 [==============================] - 0s 50ms/step - loss: 263508544.0000 - mse: 263508544.0000 - mae: 14128.1816 - val_loss: 334189696.0000 - val_mse: 334189696.0000 - val_mae: 14966.6348
Epoch 3/100
1/1 [==============================] - 0s 57ms/step - loss: 263508096.0000 - mse: 263508096.0000 - mae: 14128.1719 - val_loss: 334189184.0000 - val_mse: 334189184.0000 - val_mae: 14966.6240
Epoch 4/100
1/1 [==============================] - 0s 62ms/step - loss: 263507616.0000 - mse: 263507616.0000 - mae: 14128.1602 - val_loss: 334188672.0000 - val_mse: 334188672.0000 - val_mae: 14966.6123
Epoch 5/100
1/1 [==============================] - 0s 57ms/step - loss: 263507152.0000 - mse: 263507152.0000 - mae: 14128.1484 - val_loss: 334188128.0000 - val_mse: 334188128.0000 - val_mae: 

Evaluation Step

In [65]:
model.evaluate(x_test,y_test)

2/2 [==============================] - 0s 7ms/step - loss: 151921584.0000 - mse: 151921584.0000 - mae: 10918.2607


[151921584.0, 151921584.0, 10918.2607421875]

Prediction

In [66]:
model.predict(x_test).sum()

148.79848